## MIMIC-III Typo Synthetic Dataset (with new lexicon)

This notebook makes an synthetic typo dataset from MIMIC-III dataset. We randomly choose words (length >= 4) from clinical notes, corrupt them.

The possible corruptions are (We do one or two corruptions for each word):
1. Adding a character
2. Deleting a character
3. Substituting a character
4. Swaping two adjacent characters

The dictionary (a set of valid words) we use here is ~from clinspell paper (`lexicon_en.json`)~ generated with the `LRWD` and the `prevariants` table of UMLS and a English dictionary from [here](https://github.com/dwyl/english-words) (12/14/2020).

The output of this notebook is a dataset of (context, typo, answer) in TSV format.

In [1]:
import os
import sys
import re
import csv
import shutil
import random
import multiprocessing
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from tokenise import tokenize
from collections import defaultdict, Counter
from mimic_conv_notes import clean_text, sanitize_text

In [2]:
# Data path
mimic_note_fpath = os.path.expanduser('~/workspace/mimic3/csv/NOTEEVENTS.csv')  # MIMIC-III
mimic_tools_dpath = os.path.expanduser('~/workspace/mimic-tools/')  # Pseudonymization
clinspell_lexicon_fpath = os.path.expanduser('../data/lexicon/lexicon.json')  # Dictionary

# Output path
data_root = '../data/mimic_synthetic/'
# data_root = '../data/mimic_synthetic_trans/'
num_val_examples = 10000
num_test_examples = 10000
num_examples = num_val_examples + num_test_examples
all_output_fpath = os.path.join(data_root, 'all.tsv')
val_output_fpath = os.path.join(data_root, 'val.tsv')
test_output_fpath = os.path.join(data_root, 'test.tsv')
correct_min_length = 5
no_corruption_prob = 0.1
max_corruptions = 2
do_substitution = True
do_transposition = True

temp_root = '/tmp/typot'
pseudo_in_dpath = os.path.join(temp_root, 'temp')
pseudo_out_dpath = os.path.join(temp_root, 'temp_pseudonym')

## Load data

In [3]:
# Read MIMIC-III notes
print(f'Read {os.path.basename(mimic_note_fpath)}... ', end='')
df_notes = pd.read_csv(mimic_note_fpath, low_memory=False)
df_notes = df_notes.set_index('ROW_ID')
print(f'done! {len(df_notes)} notes')

Read NOTEEVENTS.csv... done! 2083180 notes


In [4]:
# Load & preprocess clinspell lexicon
print(f'Read {clinspell_lexicon_fpath}... ', end='')
with open(clinspell_lexicon_fpath, 'r') as fd:
    vocab = json.load(fd)
vocab_set = set(vocab)
print(f'{len(vocab)} words')

Read ../data/lexicon/lexicon.json... 822919 words


## Select notes & corrupt words

In [5]:
# Choose words
def random_word_context(text):
    words = text.split()
    while True:
        w_idx = random.randint(0, len(words)-1)
        word = words[w_idx]
        
        if word[-1] in ['.', ',']:
            word, res = word[:-1], [word[-1]]
        else:
            word, res = word, []
            
        if len(word) <= 3 or (not word.lower() in vocab_set):
            continue
            
        for i in range(0, -4, -1):
            if w_idx+i >= 0 and '[**' in words[w_idx+i]:
                continue
        for i in range(0, 4):
            if w_idx+i < len(words) and '**]' in words[w_idx+i]:
                continue
        break
    return word, ' '.join(words[:w_idx]), ' '.join(res + words[w_idx+1:])

In [6]:
# Corrupt words
alphabet = 'abcdefghijklmnopqrstuvwxyz'
def random_alphabet():
    return alphabet[random.randint(0, len(alphabet)-1)]

operation_list = ['ins', 'del']
if do_substitution:
    operation_list.append('sub')
if do_transposition:
    operation_list.append('tra')

def single_corruption(word):
    oper = random.choice(operation_list)
    if oper == 'del':  # deletion
        while True:
            cidx = random.randint(0, len(word)-1)
            if word[cidx].isalpha():  # Only delete an alphabet
                ret = word[:cidx] + word[cidx+1:]
                break
    elif oper == 'ins':  # insertion
        cidx = random.randint(0, len(word))
        ret = word[:cidx] + random_alphabet() + word[cidx:]
    elif oper == 'sub':  # substitution
        while True:
            cidx = random.randint(0, len(word)-1)
            if word[cidx].isalpha():  # Only replace an alphabet
                c = random_alphabet()
                if c != word[cidx]:
                    ret = word[:cidx] + c + word[cidx+1:]
                    break
    elif oper == "tra":  # transposition
        cidx = random.randint(0, len(word)-2) # swap cidx-th and (cidx+1)-th char
        ret = word[:cidx] + word[cidx+1] + word[cidx] + word[cidx+2:]        
    else:
        raise ValueError(f'Wrong operation {oper}')
    return ret

def corrupt_word(word_original, max_corruptions=2):
    if no_corruption_prob > 0.0:
        if random.uniform(0, 1) < no_corruption_prob:
            return word_original

    num_corruption = random.randint(1, max_corruptions)
    while True:
        word = word_original
        for i in range(num_corruption):
            word = single_corruption(word)
        if word_original != word:
            break
    return word

In [7]:
# Select note indexes randomly
random.seed(1234)
note_ids = list(df_notes.index)
random.shuffle(note_ids)

count, typo_noteids = 0, set()
for nid in note_ids:
    note = df_notes.loc[nid].TEXT
    if len(note.strip()) >= 2000 and nid not in typo_noteids:  # Only choose for len(text) >= 1000
        typo_noteids.add(nid)
        count += 1
    if count == num_examples:
        break

typo_noteids = list(typo_noteids)
print(typo_noteids[:10])

[655360, 393217, 524290, 393216, 393224, 16, 393244, 1572897, 34, 786477]


In [8]:
# Select words
examples = []
for nid in tqdm(typo_noteids):
    text = df_notes.loc[nid].TEXT
    word, left, right = random_word_context(text)
    examples.append([word, left, right])

100%|██████████| 20000/20000 [00:05<00:00, 3887.16it/s]


In [9]:
# Write notes with misspells
if os.path.exists(pseudo_in_dpath):
    shutil.rmtree(pseudo_in_dpath)        
if os.path.exists(pseudo_out_dpath):
    shutil.rmtree(pseudo_out_dpath)
    
os.makedirs(pseudo_in_dpath)
for noteid, (_, left, right) in zip(typo_noteids, examples):
    with open(os.path.join(pseudo_in_dpath, f'{noteid}_left.txt'), 'w', encoding='utf-8') as fd:
        fd.write(left)
    with open(os.path.join(pseudo_in_dpath, f'{noteid}_right.txt'), 'w', encoding='utf-8') as fd:
        fd.write(right)

In [10]:
# pip install requests joblib sqlalchemy gensim
! python {os.path.join(mimic_tools_dpath, 'main.py')} REPLACE \
    --input-dir {os.path.join(os.getcwd(), pseudo_in_dpath)} \
    --output-dir {os.path.join(os.getcwd(), pseudo_out_dpath)} \
    --list-dir {os.path.join(mimic_tools_dpath, 'lists')}

2021-01-04 00:40:04,693 Starting placeholder replacing
2021-01-04 00:40:04,694 Loading lists
2021-01-04 00:40:04,713 * Postal addresses: 20000 [656C Newport Court Coatesville, PA 19320 ...]
2021-01-04 00:40:04,900 * Last names: 88799 [SMITH, JOHNSON, WILLIAMS, JONES, BROWN ...]
2021-01-04 00:40:04,903 * Male first names: 1219 [JAMES, JOHN, ROBERT, MICHAEL, WILLIAM ...]
2021-01-04 00:40:04,912 * Female first names: 4275 [MARY, PATRICIA, LINDA, BARBARA, ELIZABETH ...]
2021-01-04 00:40:04,930 * Phone numbers: 20000 [(666) 372-7835, (923) 739-2644 ...]
2021-01-04 00:40:04,949 * Companies: 20000 [Ligula Aenean Gravida Ltd, Non Bibendum Sed LLC ...]
2021-01-04 00:40:04,949 * Countries: 264 [Afghanistan, Albania, Algeria, American Samoa ...]
2021-01-04 00:40:04,968 * Emails: 20000 [enim.Suspendisse.aliquet@Crasdictum.com, sapien.Cras.dolor@Curabitur.org ...]
2021-01-04 00:40:04,969 * Holiday names: 187 [Administrative Professionals Day, Air Force Birthday ...]
2021-01-04 00:40:04,973 * Hospit

In [11]:
# Read pseudonymized notes
def process_note(note):
    note = re.sub('\n', ' ', note)
    note = re.sub('\t', ' ', note)
    note = sanitize_text(clean_text(note))
    return note

for nid, example in tqdm(zip(typo_noteids, examples), total=len(typo_noteids)):
    with open(os.path.join(pseudo_out_dpath, f'{nid}_left.txt'), 'r', encoding='utf-8') as fd:
        note = fd.read()
        note = process_note(note)
        example[1] = note
    with open(os.path.join(pseudo_out_dpath, f'{nid}_right.txt'), 'r', encoding='utf-8') as fd:
        note = fd.read()
        note = process_note(note)
        example[2] = note
    example[0] = example[0].lower()

100%|██████████| 20000/20000 [00:21<00:00, 919.97it/s] 


In [12]:
# Corrupt words
print('Generate corrupted words... ')
random.seed(1234)
correct_words = [e[0] for e in examples]
typo_words = [corrupt_word(w, max_corruptions) for w in correct_words]
for i, (w1, w2) in enumerate(zip(correct_words, typo_words)):
    print(f'\t{w1} -> {w2}')
    if i == 5: break
print('done!')

Generate corrupted words... 
	examination -> cexaminatiwon
	atrial -> atrial
	mmhg -> zmmhg
	surgical -> surgical
	ulcer -> uwcler
	gastroesophageal -> gastroesophagela
done!


## Generate dataset for BERT

Data format (TSV): `index`,`note_id`,`word`,`left`,`right`,`correct`

- `index`: index of the data (starting from 0)
- `note_id`: ROW_ID of MIMIC-III `NOTEEVENTS.csv`
- `word`: the word of interest (typo)
- `left`: left context
- `right`: right context
- `correct`: correction

In [13]:
random.seed(1234)
data_split_idx = list(range(num_examples))
random.shuffle(data_split_idx)
val_idx, test_idx = data_split_idx[:num_val_examples], data_split_idx[num_val_examples:]
val_idx.sort()
test_idx.sort()

In [14]:
if not os.path.exists(data_root):
    os.makedirs(data_root)

for fpath, idx_list in [(all_output_fpath, list(range(num_examples))),
                   (val_output_fpath, val_idx),
                   (test_output_fpath, test_idx)]:
    print(f'Write examples to {fpath}... ', end='', flush=True)
    with open(fpath, 'w') as fd:
        writer = csv.writer(fd, delimiter='\t')
        writer.writerow(['index', 'note_id', 'word', 'left', 'right', 'correct'])  
        for i in idx_list:
            nid, (correct, left, right), typo = typo_noteids[i], examples[i], typo_words[i]
            left = ' '.join(left.split(' ')[-128:])
            right = ' '.join(right.split(' ')[:128])
            line = [i, nid, typo, left, right, correct]
            writer.writerow(line)
    print('done!')

Write examples to ../data/mimic_synthetic/all.tsv... done!
Write examples to ../data/mimic_synthetic/val.tsv... done!
Write examples to ../data/mimic_synthetic/test.tsv... done!
